# Семинар 6. Основы работы с библиотеками scanpy и Seurat

Это практикум с анализом данных scRNA-Seq. Он основан на пакете scanpy для Python 3 и Seurat для R версии 4 и больше.

## Домашнее задание

Сделайте несколько вспомогательных функций, которые вам позднее пригодятся:
1. Разбиение одного объекта AnnData на список других, соответствующих каждому индивидуальному батчу;
2. Создание нового объекта со случайными N клетками и M генами из изначального;
3. Сохранение вашего AnnData-объекта как Seurat-объекта в формате `.rds`.

In [1]:
import scanpy as sc
import numpy as np
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

adata = sc.read_10x_mtx("filtered_feature_bc_matrix") # Эта функция позволяет быстро прочитать аутпут программы Cell Ranger
adata = adata[:, (adata.X > 0).sum(axis=0).A[0] > 3] #гены, которые экспрессируются в >3 клетках
adata = adata[adata.X.sum(axis=1) > 10000] #клетки, в которых суммарная экспрессия > 10000 молекул
adata.obs['batch'] = ["1"] * 1000 + ["2"] * 1944

/tmp/ipykernel_814709/4206433042.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['batch'] = ["1"] * 1000 + ["2"] * 1944


In [29]:
# 1-ая функция
def split_adata(adata):
    adata_dict = {i:adata[adata.obs.batch == i] for i in adata.obs.batch.unique()}
    return adata_dict

In [115]:
# 2-ая функция
def random_adata(adata, N, M):
    if isinstance(adata, sc.AnnData):
        adata_new = adata[np.random.choice(adata.shape[0]+1, size=N), :] # N клеток
        adata_new = adata_new[:, np.random.choice(adata_new.shape[1]+1, size=M)] # M генов
        return adata_new
    else: 
        raise TypeError('Not AnnData datatype')

In [6]:
# 3-яя функция
def save_as_seurat(adata, path='./'):
    import rpy2.robjects as ro
    ro.r("library(Seurat)")
    ro.globalenv['adata_raw'], ro.globalenv['path'] = adata, path 
    ro.r("adata_to_seurat <- as.Seurat(adata_raw, data = NULL, counts = 'X')")
    ro.r("saveRDS(adata_to_seurat, paste(path,'SeuratObj.rds',sep=''))")